In [21]:
# Análisis Exploratorio de Datos - Vehículos Usados

import pandas as pd
import plotly.express as px
import streamlit as st

# 1. Cargar datos
car_data = pd.read_csv('../vehicles_us.csv')
print("Dimensiones del dataset:", car_data.shape)
print("\nPrimeras 5 filas:")
print(car_data.head())

# 2. Información básica
print("\nInformación del dataset:")
print(car_data.info())

print("\nColumnas disponibles:")
print(car_data.columns.tolist())

# 3. Estadísticas descriptivas
print("\nEstadísticas descriptivas:")
print(car_data.describe())

# 4. Valores nulos
print("\nValores nulos por columna:")
print(car_data.isnull().sum())

# 5. Análisis de precios (ejemplo)
print("\nAnálisis de precios:")
print(f"Precio promedio: ${car_data['price'].mean():,.2f}")
print(f"Precio máximo: ${car_data['price'].max():,.2f}")
print(f"Precio mínimo: ${car_data['price'].min():,.2f}")

# 6. Gráfico de ejemplo (opcional)
# fig = px.histogram(car_data, x='price', title='Distribución de Precios')
# fig.show()

Dimensiones del dataset: (51525, 13)

Primeras 5 filas:
   price  model_year           model  condition  cylinders fuel  odometer  \
0   9400      2011.0          bmw x5       good        6.0  gas  145000.0   
1  25500         NaN      ford f-150       good        6.0  gas   88705.0   
2   5500      2013.0  hyundai sonata   like new        4.0  gas  110000.0   
3   1500      2003.0      ford f-150       fair        8.0  gas       NaN   
4  14900      2017.0    chrysler 200  excellent        4.0  gas   80903.0   

  transmission    type paint_color  is_4wd date_posted  days_listed  
0    automatic     SUV         NaN     1.0  2018-06-23           19  
1    automatic  pickup       white     1.0  2018-10-19           50  
2    automatic   sedan         red     NaN  2019-02-07           79  
3    automatic  pickup         NaN     NaN  2019-03-22            9  
4    automatic   sedan       black     NaN  2019-04-02           28  

Información del dataset:
<class 'pandas.core.frame.DataFrame

In [ ]:
# Importación de todas las librerías 
import pandas as pd
import plotly.express as px
import streamlit as st

# Leer los datos del archivo csv
car_data = pd.read_csv('../vehicles_us.csv')

# Título de la aplicación 
st.title("Análisis de Vehículos Usados")

#Visualizador de datos del DataFrame

#creamos una pequeña sección en la que podemos visualizar la información básica de nuestro csv
st.header("Información de DataFrame")
col1, col2, col3 = st.columns(3) #creamos 3 columnas horizontales y cada coln es la variable que representa cada columna 
with col1:
    st.metric("Total de vehículos", f"{len(car_data):,}") # Nos ayuda a crear una métrica visual sobre las filas que tiene el dataframe
with col2:
    st.metric("Total de columnas", len(car_data.columns)) # Devuelve la lista con los nombre de las columnas  y las cuenta
with col3:
    st.metric("Años cubiertos", 
              f"{int(car_data['model_year'].min())}-{int(car_data['model_year'].max())}") # Accede al modelo del coche, lo convierte en entero y muestra del año más viejo al más nuevo

# Ahora para ver todos los datos y filtrarlos de diferente forma
st.header("Visualizar los Datos")

# Usamos los tabs para tener diferentes vistas de nuestros datos
tab1, tab2, tab3 = st.tabs(["Dataset Completo", "Primeras Filas", "Muestra Aleatoria"]) #cada tab es una pestañaa creada en el interfaz.

# Muestra el Dataframe completo 
with tab1:
    st.write("Lista de vehículos:")
    st.dataframe(car_data)

# En la pestaaña 2 añadimos las primeras 15 filas 
with tab2:
    st.write("Primeras 15 filas:")
    st.dataframe(car_data.head(15))

# EN la pestaña 3 añadimos 20 filas aleatorias
with tab3:
    st.write("**Muestra aleatoria de 20 filas:**")
    st.dataframe(car_data.sample(20))

st.header("Resumen Estadístico")
# Añadimos las estadísticas generales que nos aporta le método describe()
if st.checkbox("Mostrar estadísticas descriptivas:"):
    st.write(car_data.describe())

# Gráfico para tipos de vehículo por fabricante
st.header("Tipos de Vehículos por Fabricante")

# Creamos una columna temporal para el fabricante en donde lo que se extrae es el modelo
car_data["manufacturer"] = car_data["model"].str.split().str[0]

# Opciones que puede configurar el usuario
st.subheader("Configurar el gráfico")

# Opción para Seleccionar fabricantes
manufacturers = sorted(car_data['manufacturer'].dropna().unique())
selected_manufacturers = st.multiselect(
    "Selecciona fabricantes:",
    options=manufacturers,
    default=manufacturers[:5] if len(manufacturers) > 5 else manufacturers
)

# Opción para seleccionar tipos de vehículos
vehicle_types = sorted(car_data['type'].dropna().unique())
selected_types = st.multiselect(
    "Selecciona tipos de vehículos:",
    options=vehicle_types,
    default=vehicle_types[:5] if len(vehicle_types) > 5 else vehicle_types
)

# Opción para el orden del gráfico
sort_order = st.radio(
    "Ordenar por:",
    ["Conteo total (descendente)", "Alfabéticamente (fabricante)"],
    horizontal=True
)

# Opción para grupar por
group_by = st.radio(
    "Agrupar por:",
    ["Fabricante", "Tipo de vehículo"],
    horizontal=True
)

# Botón para generar el gráfico
if st.button("Generar gráfico de barras") or (len(selected_manufacturers) > 0 and len(selected_types) > 0):
    
    # Filtrar datos según selección
    filtered_data = car_data[
        (car_data['manufacturer'].isin(selected_manufacturers)) &
        (car_data['type'].isin(selected_types))
    ]
    
    if len(filtered_data) == 0:
        st.warning("No hay datos para las selecciones realizadas.")
    else:
        # Crear el gráfico según la opción de agrupación
        if group_by == "Fabricante":
            # Agrupar por fabricante y tipo
            pivot_data = filtered_data.groupby(['manufacturer', 'type']).size().reset_index(name='count')
            
            # Crear gráfico de barras agrupadas
            fig = px.bar(
                pivot_data,
                x='manufacturer',
                y='count',
                color='type',
                title='Tipos de Vehículos por Fabricante',
                labels={
                    'manufacturer': 'Fabricante',
                    'count': 'Número de Vehículos',
                    'type': 'Tipo de Vehículo'
                },
                barmode='group'  # Barras agrupadas
            )
            
        else:  # Agrupar por tipo de vehículo
            # Agrupar por tipo y fabricante
            pivot_data = filtered_data.groupby(['type', 'manufacturer']).size().reset_index(name='count')
            
            # Crear gráfico de barras agrupadas
            fig = px.bar(
                pivot_data,
                x='type',
                y='count',
                color='manufacturer',
                title='Fabricantes por Tipo de Vehículo',
                labels={
                    'type': 'Tipo de Vehículo',
                    'count': 'Número de Vehículos',
                    'manufacturer': 'Fabricante'
                },
                barmode='group'
            )
        
        # Ordenar según selección
        if sort_order == "Conteo total (descendente)":
            if group_by == "Fabricante":
                # Ordenar fabricantes por conteo total
                manufacturer_totals = filtered_data.groupby('manufacturer').size()
                sorted_manufacturers = manufacturer_totals.sort_values(ascending=False).index
                fig.update_xaxes(categoryorder='array', categoryarray=sorted_manufacturers)
            else:
                # Ordenar tipos por conteo total
                type_totals = filtered_data.groupby('type').size()
                sorted_types = type_totals.sort_values(ascending=False).index
                fig.update_xaxes(categoryorder='array', categoryarray=sorted_types)
        
        # Personalizar el gráfico
        fig.update_layout(
            height=500,
            showlegend=True,
            legend_title_text='Leyenda',
            xaxis_tickangle=-45 if len(selected_manufacturers) > 5 else 0
        )
        
        # Mostrar el gráfico
        st.plotly_chart(fig, use_container_width=True)
        
        # Mostrar datos de respaldo
        with st.expander("Ver datos del gráfico"):
            st.write("Conteo por categoría:")
            st.dataframe(pivot_data)
            
            # Estadísticas
            col1, col2, col3 = st.columns(3)
            with col1:
                st.metric("Total vehículos mostrados", len(filtered_data))
            with col2:
                st.metric("Fabricantes mostrados", len(selected_manufacturers))
            with col3:
                st.metric("Tipos mostrados", len(selected_types))


2025-12-04 15:56:04.811 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 15:56:04.814 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 15:56:04.816 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 15:56:04.820 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 15:56:04.822 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 15:56:04.824 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 15:56:04.827 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 15:56:04.829 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [18]:
# Creación de Histograma de su condición vs año del modelo
st.header("Histograma: Condición vs Año del Modelo")

# Prepara los datos y limpia los años nulos si es que los hay
car_data_clean = car_data.dropna(subset=["model_year", "condition"])
car_data_clean["model_year"] = car_data_clean["model_year"].astype(int)

# Información sobre los datos 
st.write(f"Vehículos con año y condición válidos: {len(car_data_clean):,} de {len(car_data):,}")

# Controles para el histograma 
st.subheader("COnfiguración del histograma")

# Opción para seleccionar el rango de años
min_year = int(car_data_clean["model_year"].min())
max_year = int(car_data_clean["model_year"].max())

year_range = st.slider("Selecciona rango de años:",
                       min_value = min_year,
                       max_value = max_year, 
                       value = (min_year, max_year),
                       step = 1)

# Opción para seleccionar las condiciones que debe de mostrar 
conditions = sorted(car_data_clean["condition"].dropna().unique())
# Para traducir nuestras condiciones al español y se vea mas limpio seeún sus variables
condition_translation = {"excellent": "Excelente",
                         "good": "Bueno",
                         "like new": "Como Nuevo",
                         "fair": "Regular",
                         "salvage": "Siniestro"}
selected_conditions = st. multiselect( "Selecciona condiciones a mostrar",
                                      options=conditions,
                                      default=conditions,
                                      format_func=lambda x: condition_translation.get(x, x))

# Opcion para el ancho de la barra
bin_size = st.slider("Ancho de las barras (años por barra):",
                     min_value=1,
                     max_value=5,
                     value=1,
                     step=1,
                     help="Controla cuantos años agrupar en  cada barra")

#Opción para personalizar el histograma
hist_type = st.radio("Tipo de histograma:",
                     ["Barras agrupadas", "Barras apiladas"],
                     horizontal=True)

# Botón para generar histograma 
if st.button("Generar histograma de condición") or (len(selected_conditions) > 0):
    # Filtro de datos
    filtered_data = car_data_clean[(car_data_clean["model_year"] >= year_range[0]) & (car_data_clean["model_year"] <= year_range[1]) & (car_data_clean["condition"].isin(selected_conditions))]

    if len(filtered_data) ==0:
        st.warning("No hay datos para los filtros seleccionados.")
    else:
        # Creación del histograma
        if hist_type == "Barras agrupadas":
            fig = px.histogram(filtered_data, 
                               x= "model_year",
                               color = "condition",
                               title = "Histograma: Condición vs Año del Modelo",
                               labels = {"model_year": "Año del Modelo",
                                         "count": "Número de Vehículos",
                                         "condition": "Condición"},
                                nbins= ((year_range[1] - year_range[0]) // bin_size) +1,
                                barmode="group", 
                                color_discrete_map={'excellent': '#2E86AB',    # Azul
                                                    'good': '#A23B72',         # Púrpura
                                                    'like new': '#F18F01',     # Naranja
                                                    'fair': '#C73E1D',         # Rojo
                                                    'salvage': '#6A994E'       # Verde
                                                    }
                                )
        else:
            fig = px.histogram(filtered_data,
                               x= "model_year",
                               color = "condition",
                               title = "Histograma: Condición vs Año del Modelo",
                               labels = {
                                   'model_year': 'Año del Modelo',
                                   'count': 'Número de Vehículos',
                                   'condition': 'Condición'},
                                 nbins=((year_range[1] - year_range[0]) // bin_size) + 1,
                                 barmode='stack',  # Barras apiladas
                                 color_discrete_map={
                                     'excellent':'#2E86AB',
                                     'good': '#A23B72', 
                                     'like new': '#F18F01',
                                     'fair': '#C73E1D',
                                     'salvage': '#6A994E'
                                     },
                                     category_orders={
                                         'condition': ['excellent', 'good', 'like new', 'fair', 'salvage']
                }
                                   )
                
                # Personalizar el gráfico
        fig.update_layout(
            height=500,
            showlegend=True,
            legend_title_text='Condición',
            xaxis=dict(
                title='Año del Modelo',
                tickmode='linear',
                dtick=1 if (year_range[1] - year_range[0]) <= 20 else 2,
                tickangle=-45 if (year_range[1] - year_range[0]) > 10 else 0
            ),
            yaxis=dict(
                title='Número de Vehículos'
            ),
            hovermode='x unified'  # Muestra todos los valores al pasar el mouse
        )
        
        # Actualizar nombres en la leyenda
        fig.for_each_trace(lambda trace: trace.update(
            name=condition_translation.get(trace.name, trace.name),
            legendgroup=condition_translation.get(trace.name, trace.name)
        ))
        
        # Mostrar el gráfico
        st.plotly_chart(fig, use_container_width=True)
        
        # Análisis adicional
        with st.expander("Análisis de los datos"):
            
            # Estadísticas por condición
            st.subheader("Resumen por condición")
            condition_stats = filtered_data['condition'].value_counts()
            
            # Crear columnas para métricas
            cols = st.columns(len(condition_stats))
            for idx, (condition, count) in enumerate(condition_stats.items()):
                with cols[idx]:
                    st.metric(
                        condition_translation.get(condition, condition),
                        f"{count:,}",
                        f"{count/len(filtered_data)*100:.1f}%"
                    )
            
            # Tabla detallada
            st.subheader("Distribución detallada")
            year_condition_pivot = filtered_data.pivot_table(
                index='model_year',
                columns='condition',
                aggfunc='size',
                fill_value=0
            )
            
            # Renombrar columnas en español
            year_condition_pivot.columns = [
                condition_translation.get(col, col) for col in year_condition_pivot.columns
            ]
            
            st.dataframe(year_condition_pivot)
            
            # Insights automáticos
            st.subheader("Insights")
            
            # Año con más vehículos
            most_common_year = filtered_data['model_year'].mode()[0]
            st.write(f"Año más común: {most_common_year} ({filtered_data[filtered_data['model_year'] == most_common_year].shape[0]:,} vehículos)")
            
            # Condición más común
            most_common_condition = filtered_data['condition'].mode()[0]
            st.write(f"Condición más común: {condition_translation.get(most_common_condition, most_common_condition)}")
            
            # Evolución de condiciones por año
            st.write(f"Evolución de condiciones:")
            for condition in selected_conditions:
                condition_data = filtered_data[filtered_data['condition'] == condition]
                if len(condition_data) > 0:
                    avg_year = condition_data['model_year'].mean()
                    st.write(f"- {condition_translation.get(condition, condition)}: Año promedio {avg_year:.0f}")


with st.expander("Guía de condiciones"):
    st.write("""
    **Significado de las condiciones:**
    
    - **Excelente**: Vehículo en perfecto estado, como nuevo
    - **Bueno**: Vehículo en buen estado, con uso normal
    - **Como Nuevo**: Vehículo casi sin usar, prácticamente nuevo
    - **Regular**: Vehículo con desgaste visible pero funcional
    - **Siniestro**: Vehículo con daños significativos o reconstruido
    """)
    
    # Distribución general de condiciones
    st.write("Distribución general de condiciones en todo el dataset:")
    condition_dist = car_data['condition'].value_counts()
    
    # Traducir etiquetas
    condition_dist.index = [condition_translation.get(idx, idx) for idx in condition_dist.index]
    
    fig_pie = px.pie(
        values=condition_dist.values,
        names=condition_dist.index,
        title="Distribución de condiciones",
        color_discrete_sequence=px.colors.qualitative.Set2
    )
    fig_pie.update_traces(textposition='inside', textinfo='percent+label')
    st.plotly_chart(fig_pie, use_container_width=True)

2025-12-04 21:19:49.579 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 21:19:49.589 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 21:19:49.594 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
C:\Users\arivl\AppData\Local\Temp\ipykernel_31056\2202733229.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

2025-12-04 21:19:49.641 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 21:19:49.643 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 21

In [19]:
# Gráfico de dispersión 

st.header("Gráfico de Dispersión: Año vs Precio")

#Botón para el gráfico de dispersión 

scatter_botton = st.button("Construit gráfico de dispersión")

if scatter_botton:
    st.write("Creación de un gráfico de dispersión para el conjunto de datos de anuncios de venta de coches")

    # LImpiar datos para el scatter plot
    scatter_data = car_data.dropna(subset = ["model_year", "price"])
    scatter_data["model_year"] = scatter_data["model_year"].astype(int)

    #Creación de ráfico de dispersión
    fig = px.scatter( scatter_data,
                     x = "model_year",
                     y = "price",
                     title = "Relación entre Año del modelo y precio",
                     labels = {
            'model_year': 'Año del Modelo',
            'price': 'Precio (USD)'
        },
        opacity = 0.6,
        trendline = "ols",
        trendine_color_override = "red")
    
    # Para personalizar el gráfico 
    fig.update_layout(
        height=500,
        xaxis=dict(
            tickmode='linear',
            dtick=1
        ),
        yaxis=dict(
            tickprefix='$',
            tickformat=',.0f'
        )
    )
    
    # Mostrar gráfico
    st.plotly_chart(fig, use_container_width=True)
    
    # Estadísticas de correlación
    correlation = scatter_data['model_year'].corr(scatter_data['price'])
    st.write(f"**Correlación entre año y precio:** {correlation:.2f}")
    st.write("*(1 = correlación perfecta positiva, -1 = perfecta negativa, 0 = no correlación)*")

# Nota importante: Apliqué dropna ya que los datos inventados pueden sergar el análisis e intentandolos correr no me dejaba, por lo tanto los tuve que eliminar.

2025-12-04 21:39:05.611 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 21:39:05.614 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 21:39:05.616 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 21:39:05.621 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 21:39:05.622 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 21:39:05.625 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 21:39:05.628 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 21:39:05.629 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar